# 分岐点の半径Rのラベルを削除して実行
分岐点の半径Rの円内にあるスペクトラルクラスタリングのラベルを削除する
#### 入力
- スペクトラルクラスタリングにより20分割した画像

### ライブラリのインポート

In [ ]:
%matplotlib inline

import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
import datetime
import math
from skimage.morphology import skeletonize
import random

### 細線化により分岐点探索

In [ ]:
def junk(img_skel,img,img_num):
    height,width = img_skel.shape
    temp_j = np.zeros((height,width))
    
    skel_b = img.copy()
    
    for y in range(height):
        for x in range(width):
            if(img_skel[y,x]>0):
                skel_b[y,x] = (255,255,255)
            else:
                skel_b[y,x] = (0,0,0)
    
    for y in range(height):
        for x in range(width):
            if(img_skel[y,x]==255):
                check = 0
                cnt = 1
                temp_l = np.zeros((3,3))
                
                if(img_skel[y-1,x-1]==255):
                    cnt += 1
                    temp_l[0,0] = 1
                if(img_skel[y-1,x]==255):
                    cnt += 1
                    temp_l[0,1] = 1
                if(img_skel[y-1,x+1]==255):
                    cnt += 1
                    temp_l[0,2] = 1
                if(img_skel[y,x-1]==255):
                    cnt += 1
                    temp_l[1,0] = 1
                if(img_skel[y,x+1]==255):
                    cnt += 1
                    temp_l[1,2] = 1
                if(img_skel[y+1,x-1]==255):
                    cnt += 1
                    temp_l[2,0] = 1
                if(img_skel[y+1,x]==255):
                    cnt += 1
                    temp_l[2,1] = 1
                if(img_skel[y+1,x+1]==255):
                    cnt += 1
                    temp_l[2,2] = 1
                
                for y_l in range(3):
                    for x_l in range(3):
                        if(temp_l[y_l,x_l] == 1):
                            if((y_l-1)>=0):
                                if(temp_l[y_l-1,x_l] == 1):
                                    check = 1
                            if((y_l+1)<3):
                                if(temp_l[y_l+1,x_l] == 1):
                                    check = 1
                            if((x_l-1)>=0):
                                if(temp_l[y_l,x_l-1] == 1):
                                    check = 1
                            if((x_l+1)<3):
                                if(temp_l[y_l,x_l+1] == 1):
                                    check = 1

                if(cnt == 2):
                    temp_j[y,x] = 2
                elif((cnt > 3) and (check == 0)):
                    temp_j[y,x] = 1
                elif(cnt>4):
                    temp_j[y,x] = 1
                    
    for y in range(height):
        for x in range(width):
            if(temp_j[y,x]==1):
                cv2.rectangle(img, (x-3, y-3), (x+3, y+3), (0, 0, 255))
                cv2.rectangle(skel_b, (x-3, y-3), (x+3, y+3), (0, 0, 255))
                cv2.rectangle(img_skel, (x-3, y-3), (x+3, y+3), 0, thickness=-1)
            elif(temp_j[y,x]==2):
                cv2.rectangle(img, (x-3, y-3), (x+3, y+3), (0, 255, 0))
                
    #cv2.imwrite('Output/Spectral_sys/03_skel-junktion/ORA-{0:03d}_skel-junction.png'.format(img_num),skel_b)
                    
    return temp_j,img,img_skel

### 細線化

In [ ]:
def skel(img_num):

    input_img_file = '../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif'.format(img_num)
    
    #Original
    img_col = cv2.imread(input_img_file)

    #NLMDを用いた2値化
    img = cv2.imread(input_img_file,0)
    img = cv2.fastNlMeansDenoising(img,h=6)
    th_img = np.where(img>50 ,1 ,0)

    # 細線化(スケルトン化) Zha84
    skeleton = skeletonize(th_img)

    skel_b = np.zeros((img.shape[0],img.shape[1]))
    skel_b[skeleton==True] = 255
    #cv2.imwrite('Output/Spectral_sys/02_skeleton/ORA-{0:03d}_skeleton.png'.format(img_num),skel_b)

    #細線化画像による分岐点探索
    temp_j,img_j,img_skel = junk(skel_b,img_col,img_num)
    
    #cv2.imwrite('Output/Spectral_sys/04_img-junction/ORA-{0:03d}_img-junktion.png'.format(img_num),img_j)
    
    return temp_j,img_skel

### 全体のコード

In [ ]:
for img_num in tqdm(range(578,850)):
#for img_num in tqdm(range(600,601)):
    #ファイルの指定
    input_img_file = '../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif'.format(img_num)
    input_SpCl_file = 'Output/Spectral_sys/01_k-means/ORA-{0:03d}_k-means-200.png'.format(img_num)
    
    #クラスタ数
    k_c = 200
    
    #分岐点除去の半径を指定
    r = 15
    
    #入力画像
    img = cv2.imread(input_img_file,0)
    img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
    th_img = np.where(img_NLMD>50 ,255,0)
    
    #ノード数カウント用
    cnt = 1
    
    #ノード番号を格納する配列を定義
    temp = np.zeros((height,width),dtype = 'i4')

    #閾値以上のピクセルにノード番号付与
    for y in tqdm(range(height)):
        for x in range(width):
            if(th_img[y,x]==255):
                temp[y,x] = cnt
                cnt += 1
    
    now = datetime.datetime.now()
    print(now)
    print('スペクトラルクラスタリングの結果を読み込み')
    
    #スペクトラルクラスタリングの結果をインポート
    img_kmeans = cv2.imread(input_SpCl_file)
    height,width,c = img_kmeans.shape
    
    #クラスタ番号を格納する配列
    cluster = np.zeros((height,width))
    #クラスタごとの色を管理する配列
    color_code = np.zeros((300,3))
    
    #クラスタ番号のカウント用
    i = 1
    
    #クラスタ番号の付与
    for y in range(height):
        for x in range(width):
            if((img_kmeans[y,x,0]== img_kmeans[y,x,1]) and (img_kmeans[y,x,1] == img_kmeans[y,x,2])):
                img_kmeans[y,x] = (0,0,0)
            else:
                if(i==1):
                    color_code[i] = img_kmeans[y,x]
                    cluster[y,x] = i
                    i+=1
                else:
                    flag = 0
                    j = 1

                    while(flag != 1 and j <= i):
                        if((color_code[j]==img_kmeans[y,x]).all()):
                            flag = 1
                            cluster[y,x] = j
                        else:
                            j += 1

                    if(flag == 0):
                        color_code[i] = img_kmeans[y,x]
                        cluster[y,x] = i
                        i += 1
                        
        now = datetime.datetime.now()
    print(now)
    print('細線化による分岐点探索')
    
    j_p,img_skel= skel(img_num)

    now = datetime.datetime.now()
    print(now)
    print('分岐点周辺の同一クラスタを削除')
    
    del_cluster = np.zeros((height,width))
    img_del_r = cv2.imread(input_img_file)
    
    mask = cv2.imread('reference/Dark_temp.png')
    for y in tqdm(range(height)):
        for x in range(width):
            if(j_p[y,x] == 1):
                cv2.circle(mask,(x,y),r,(255, 255, 255), thickness=-1)

    for y in range(height):
        for x in range(width):
            if(mask[y,x,0]==255):
                if(cluster[y,x]!=0):
                    p_pred = cluster[y,x]
                    cluster[y,x] = 0
                    del_cluster[y,x] = img_NLMD[y,x]
                    for y_t in range(height):
                        for x_t in range(width):
                            if(p_pred == cluster[y_t,x_t]):
                                cluster[y_t,x_t] = 0
                                del_cluster[y_t,x_t] = img_NLMD[y_t,x_t]

    """
    colors = []
    for i in range(1, 300):
        colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))
                                
    img_col = cv2.imread(input_img_file)

    for y in range(img_col.shape[0]):
        for x in range(img_col.shape[1]):
            if(temp[y,x]>0):
                if(cluster[y,x] != 0):
                    i = temp[y,x]-1
                    img_col[y,x] = colors[int(pred[i])]
    """

    #cv2.imwrite('Output/Spectral_sys/05_junk-del/ORA-{0:03d}_k-means-{1:03d}_junk-del.png'.format(img_num,k_c),img_col)

    #cv2.imwrite('Output/Spectral_sys/06_junk-only/ORA-{0:03d}_k-means-{1:03d}_junk-only.png'.format(img_num,k_c),del_cluster)
    
    now = datetime.datetime.now()
    print(now)
    print('分岐点を除去した細線化をラベリング')
    
    img_col_skel = cv2.imread(input_img_file)
    skel_col = cv2.imread(input_img_file)

    img_skel = img_skel.astype('u1')

    nLabels, labelImages_skel, data, center = cv2.connectedComponentsWithStatsWithAlgorithm(img_skel, 8, cv2.CV_16U, cv2.CCL_DEFAULT)

    colors = []

    for i in range(1, 300):
        colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))

    for y in range(height):
        for x in range(width):
            if(labelImages_skel[y,x]>0):
                img_col_skel[y,x]=colors[int(labelImages_skel[y,x])]
                skel_col[y,x]=colors[int(labelImages_skel[y,x])]
            else:
                skel_col[y,x]=(0,0,0)

    #cv2.imwrite('Output/Spectral_sys/07-1_skel-label_onIMG/ORA-{0:03d}_skel-label_onIMG.png'.format(img_num),img_col_skel)
    #cv2.imwrite('Output/Spectral_sys/07-2_skel-label/ORA-{0:03d}_skel-label.png'.format(img_num),skel_col)
    
    now = datetime.datetime.now()
    print(now)
    print('分岐点周辺を削除した画像を2値化')
    
    th_img_clu = cv2.imread(input_img_file,0)

    for y in range(height):
        for x in range(width):
            if(cluster[y,x]>0):
                th_img_clu[y,x]=255
            else:
                th_img_clu[y,x]=0

    #cv2.imwrite('Output/Spectral_sys/08_junk-del-th/ORA-{0:03d}_k-means-{1:03d}_junk-del-th.png'.format(img_num,k_c),th_img_clu)

    
    now = datetime.datetime.now()
    print(now)
    print('分岐点周辺を削除した画像をラベリング')
    
    img_col = cv2.imread(input_img_file)

    # ラベリング処理
    nLabels, labelImages, data, center = cv2.connectedComponentsWithStatsWithAlgorithm(th_img_clu, 8, cv2.CV_16U, cv2.CCL_DEFAULT)

    # オブジェクト情報を項目別に抽出
    n = nLabels - 1

    label_fix = np.zeros(nLabels)

    # オブジェクト情報を利用して15ピクセル以下のラベルを削除
    for i in range(1,nLabels):

        size = data[i,4]
        if(size<15):
            label_fix[i]=0
        else:
            label_fix[i]=i

    #色情報の定義
    colors = []
    for i in range(1, nLabels+1):
        colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))

    #2値化用の配列
    label_th = np.zeros((height,width))

    #ラベリングの色を付与・2値化
    for y in range(height):
        for x in range(width):
            if((labelImages[y,x]>0)and(label_fix[labelImages[y,x]]>0)):
                img_col[y,x]=colors[int(label_fix[int(labelImages[y,x])])]
                label_th[y,x]=255
            else:
                cluster[y,x]=0

    #ラベリング結果を保存
    #cv2.imwrite('Output/Spectral_sys/09_junk-del-label/ORA-{0:03d}_k-means-{1:03d}_junk-del-label.png'.format(img_num,k_c),img_col)
    #ノイズ除去した画像を保存
    #cv2.imwrite('Output/Spectral_sys/10_junk-del-label-th/ORA-{0:03d}_k-means-{1:03d}_junk-del-label-th.png'.format(img_num,k_c),label_th)
    
    now = datetime.datetime.now()
    print(now)
    print('ラベリングを統合（細線化を用いて）')
    
    #クラスタ統合用
    c_num = np.zeros((201))

    #細線上を辿り、1つの細線に属しているクラスタを1つのクラスタに統合
    for y in tqdm(range(height)):
        for x in range(width):
            if((img_skel[y,x]>0)and(cluster[y,x]>0)):
                c_num[int(cluster[y,x])] = labelImages_skel[y,x]

    #統合後のクラスタを格納
    cluster_fix = np.zeros((height,width))

    for y in tqdm(range(height)):
        for x in range(width):
            if(cluster[y,x]>0):
                cluster_fix[y,x]= c_num[int(cluster[y,x])]

    #統合後のクラスタを保存
    img_col = cv2.imread(input_img_file)

    colors = []
    for i in range(1, 400):
        colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))

    for y in range(height):
        for x in range(width):
            if(cluster_fix[y,x]>0):
                img_col[y,x]=colors[int(cluster_fix[y,x])]

    #cv2.imwrite('Output/Spectral_sys/11_junk-del-label-inte/ORA-{0:03d}_k-means-{1:03d}_junk-del-label-inte.png'.format(img_num,k_c),img_col)
    
    now = datetime.datetime.now()
    print(now)
    print('同一クラスタが離れている場合はクラスタを分離')
    
    cnt_cl = 200
    chk_cl = np.zeros(400)

    for y in tqdm(range(height)):
        for x in range(width):
            if((cluster_fix[y,x]>0)and(chk_cl[int(cluster_fix[y,x])]==0)):

                temp_cl = np.zeros((height,width))

                for y_t in range(height):
                    for x_t in range(width):
                        if(cluster_fix[y,x]==cluster_fix[y_t,x_t]):
                            temp_cl[y_t,x_t] = 255

                temp_cl = temp_cl.astype('u1')

                nLabels_temp, labelImages_temp, data_temp, center_temp = cv2.connectedComponentsWithStatsWithAlgorithm(temp_cl, 8, cv2.CV_16U, cv2.CCL_DEFAULT)

                chk_cl[int(cluster_fix[y,x])]=1
                if(nLabels_temp>2):
                    for y_t in range(height):
                        for x_t in range(width):
                            if(labelImages_temp[y_t,x_t]>0):
                                cluster_fix[y_t,x_t] = cnt_cl+labelImages_temp[y_t,x_t]
                    cnt_cl += nLabels_temp-1

    c_num_fix = np.zeros(500)

    for y in tqdm(range(height)):
        for x in range(width):
            if((img_skel[y,x]>0)and(cluster_fix[y,x]>0)):
                c_num_fix[int(cluster_fix[y,x])] = labelImages_skel[y,x]

    cluster_fix2 = np.zeros((height,width))

    for y in tqdm(range(height)):
        for x in range(width):
            if(cluster[y,x]>0):
                cluster_fix2[y,x]= c_num_fix[int(cluster_fix[y,x])]

    img_col = cv2.imread(input_img_file)

    colors = []
    for i in range(1, 500):
        colors.append(np.array([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]))

    for y in range(height):
        for x in range(width):
            if(cluster_fix2[y,x]>0):
                img_col[y,x]=colors[int(cluster_fix2[y,x])]

    cv2.imwrite('Output/Spectral_sys/12-3_junk-del-label-inte-sp_r{0:02d}/ORA-{1:03d}_k-means-{2:03d}_junk-del-label-inte_sp_r{0:02d}.png'.format(r,img_num,k_c),img_col)

In [ ]:
cv2.imwrite('Output/Spectral_sys/12-2_junk-del-label-inte-sp_r{0:02d}/ORA-{1:03d}_k-means-200_junk-del-label-inte_sp_r{0:02d}.png'.format(r,img_num),img_col)